In [ ]:
import mdtraj as md
import numpy as np
import parmed as pmd
from warnings import filterwarnings
filterwarnings('ignore')
import logging
import os
import sys
import pickle

temperatures = [f"{T}" for T in range(300, 301, 20)]
ints = list(range(64,151,2))
# distances = [i/10 for i in ints]
distances_in_nm = [f"{d/100:.2f}" for d in ints]

output_path = f'/gibbs/arghavan/hp-results-pc/wall-LJ-energy/'
os.makedirs(output_path, exist_ok=True)

# for t in temperatures:
#     for d in distances_in_nm:
#         input_path = f'/gibbs/arghavan/counting_hbonds_between_graphite_walls/hbond_inputs/{t}K/{d}/'

In [ ]:
t = 300
d = '0.64'

input_path = f'/gibbs/arghavan/counting_hbonds_between_graphite_walls/hbond_inputs/{t}K/{d}/'
traj_path = f'{input_path}{t}K_{d}.nc'
top_path = f'{input_path}topol_edited.prmtop'

traj = md.load(traj_path, top=top_path)
tpl = traj.top
parm = pmd.load_file(top_path)

# all_ox_tpl_indices = tpl.select("name O")
# all_wat_atm_tpl_indices = tpl.select("water")
walls = tpl.select("resname =~ 'WALL'")

# n_frames = traj.n_frames
n_frames = 100

In [4]:
Acoeff = np.array(parm.parm_data['LENNARD_JONES_ACOEF'], dtype=float)
Acoeff

NameError: name 'parm' is not defined

In [ ]:

input_path = f'/gibbs/arghavan/counting_hbonds_between_graphite_walls/hbond_inputs/{t}K/{d}/'
output_path = f'/gibbs/arghavan/pair_energy/results_pair_energy/{t}K_{d}/'
mean_nbr_path = f'/gibbs/arghavan/counting_hbonds_between_graphite_walls/hbond_outputs/{t}K/{d}/all_frame_mean_neighbours.pkl'

prob_file = f'{output_path}prob_{t}K_{d}.dat'
numDens_file = f'{output_path}numDens_{t}K_{d}.dat'
dx_file = f'{output_path}E_{t}K_{d}.dx'
logfile = f'{output_path}log_{t}K_{d}.log'

traj_path = f'{input_path}{t}K_{d}.nc'
top_path = f'{input_path}topol_edited.prmtop'

traj = md.load(traj_path, top=top_path)
tpl = traj.top
parm = pmd.load_file(top_path)

all_ox_tpl_indices = tpl.select("name O")
all_wat_atm_tpl_indices = tpl.select("water")
walls = tpl.select("resname =~ 'WALL'")

n_frames = traj.n_frames
# n_frames = 50
# --------------------------- Logging Setup ---------------------------

os.makedirs(os.path.dirname(logfile), exist_ok=True)

class FlushFileHandler(logging.FileHandler):
    def emit(self, record):
        super().emit(record)
        self.flush()

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.handlers = []

file_handler = FlushFileHandler(logfile, mode='w')
file_handler.setFormatter(logging.Formatter('%(asctime)s %(message)s', datefmt='%I:%M:%S %p'))
logger.addHandler(file_handler)


# --------------------------- Constants ---------------------------

Acoeff = np.array(parm.parm_data['LENNARD_JONES_ACOEF'][0], dtype=float)
Bcoeff = np.array(parm.parm_data['LENNARD_JONES_BCOEF'][0], dtype=float)
chg_lst = np.array(parm.parm_data['CHARGE'][:4], dtype=float)*18.2223

o_charge = chg_lst[3]  
h_charge = chg_lst[1]

oo_chg = o_charge**2
hh_chg = h_charge**2
oh_chg = o_charge * h_charge


# --------------------------- Surface Between the Plates ---------------------------
wall_coords = traj.xyz[0][walls]

float_d = float(d)
x_center = 2.50
x_min, x_max = np.float32(x_center - float_d/2), np.float32(x_center + float_d/2)
y_min, y_max = np.min(wall_coords[:,1]), np.max(wall_coords[:,1])
z_min, z_max = np.min(wall_coords[:,2]), np.max(wall_coords[:,2])


radius = 3.5 #A

pairwise_energies = []
num_wat_in_box = []

for frame_num in range(n_frames):
     
    logger.info(f"Processing frame {frame_num + 1}")

    # ------------------------------ Masking ---------------------------------------
    
    all_ox_coords = traj.xyz[frame_num][all_ox_tpl_indices]
    mask = (
    (all_ox_coords[:, 0] >= x_min) & (all_ox_coords[:, 0] <= x_max) &
    (all_ox_coords[:, 1] >= y_min) & (all_ox_coords[:, 1] <= y_max) &
    (all_ox_coords[:, 2] >= z_min) & (all_ox_coords[:, 2] <= z_max)
)
    selected_ox_tpl_indices = all_ox_tpl_indices[mask]
    selected_wat_tpl_indices = [x for item in selected_ox_tpl_indices for x in range(item, item + 4)]
    index_map = {value: idx for idx, value in enumerate(selected_wat_tpl_indices)}
    num_wat_in_box.append(len(selected_ox_tpl_indices))

    # ------------------------------ Distance Matrix ---------------------------------------
    coords = traj.xyz[frame_num][selected_wat_tpl_indices]

    dist_matrix_in_nm = squareform(pdist(coords, metric='euclidean'))
    dist_matrix = dist_matrix_in_nm * 10

    # ------------------------------ Pairwise Energy Calculation ---------------------------------------
    frame_energies = [] 
    computed_pairs = set()

    for i in selected_ox_tpl_indices:
        for j in selected_ox_tpl_indices:

            pair = frozenset((i, j))

            if pair in computed_pairs:
                continue

            pair2 = frozenset((j, i))
            if pair2 in computed_pairs:
                continue

            if i < j: 
                d_i = index_map[i]
                d_j = index_map[j]

                if dist_matrix[d_i][d_j] <= radius:
                    lj_energy = (Acoeff / dist_matrix[d_i][d_j]** 12) - (Bcoeff / dist_matrix[d_i][d_j]** 6)
                    electrostatic_energy = (
                    oo_chg / dist_matrix[d_i+3][d_j+3] + oh_chg / dist_matrix[d_i+3][d_j+1] + oh_chg / dist_matrix[d_i+3][d_j+2] +
                    oh_chg / dist_matrix[d_i+1][d_j+3] + hh_chg / dist_matrix[d_i+1][d_j+1] + hh_chg / dist_matrix[d_i+1][d_j+2] +
                    oh_chg / dist_matrix[d_i+2][d_j+3] + hh_chg / dist_matrix[d_i+2][d_j+1] + hh_chg / dist_matrix[d_i+2][d_j+2]
                    )
                    total_energy = (lj_energy + electrostatic_energy) / 2
                    frame_energies.append(total_energy)
                computed_pairs.add(pair)
    pairwise_energies.extend(frame_energies)

pairwise_energies = np.array(pairwise_energies)
avg_num_wat_in_box = np.mean(num_wat_in_box)

header = f"# [gibbs input] Pairwise energy for ~ {avg_num_wat_in_box:.2f} waters: {n_frames} frames for {t}K - {float(d)*10} A simulation \n# Column: E_n (kcal/mol)"
np.savetxt(dx_file, pairwise_energies, fmt="%.6f", header=header,  delimiter=" ")
logger.info(f"Saved in {dx_file}")
logger.info(f"{n_frames} frames - plates_energies.py - plates E + Plot ")
logger.info(f"{len(pairwise_energies)} pairwise energies calculated.")
logger.info(f"{avg_num_wat_in_box:.2f} average No. of waters in the box.")


# --------------------------- Data for plotting ---------------------------

mean_nbr = []
with open(mean_nbr_path, 'rb') as f:
    while True:
        try:
            mean_nbr.append(pickle.load(f))
        except EOFError:
            break
N_nbr = mean_nbr[0][0]

bin_width = 0.06  # kcal/mol
bins = np.arange(np.floor(np.min(pairwise_energies) / bin_width) * bin_width, 
                 np.ceil(np.max(pairwise_energies) / bin_width) * bin_width + bin_width, bin_width)

counts, edges = np.histogram(pairwise_energies, bins=bins)
bin_centers = edges[:-1] + bin_width / 2

prob = counts / (bin_width * np.sum(counts))
rho_i = prob * N_nbr

with open(prob_file, "w") as f:
    for x, y in zip(bin_centers, prob):
        f.write(f"{x:.4f} {y:.6f}\n")

with open(numDens_file, "w") as f:
    for x, y in zip(bin_centers, rho_i):
        f.write(f"{x:.4f} {y:.6f}\n")